# Daily

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import coint_trading as ct
import utils as ut
import plotting as myplt
import data_io as myio
import mean_reversion as mrev

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interact_manual
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

### User Specification

In [ ]:
FROM = '2017-07-23'
name_list = ['GBPUSD Curncy', 'AUDUSD Curncy', 'NZDUSD Curncy', 'CADUSD Curncy']


TODAY = '2023-09-15'

### Read in data and perform estimation using the latest data

In [ ]:
df_fx_spot, df_fx_ivol = myio.read_all_fx_data('../data/bloomberg fx data sheet.xlsx')
df_fx_spot = df_fx_spot.loc[FROM:TODAY]
df_fx_name_list_XXXUSD = df_fx_spot[name_list]

res_estimation = ct.johansen_test_estimation(df_fx_name_list_XXXUSD, name_list, 0, 1)
res_diag = ct.johansen_test_diag(res_estimation, df_fx_name_list_XXXUSD, name_list, True, 0, 1)
weights_XXXUSD = res_estimation['weights']
basket = res_diag['Johansen Basket']

normal_vol_ts = ut.compute_daily_hist_normal_vol(basket)

df_fx_ivol = df_fx_ivol.loc[FROM:TODAY]
df_fx_ivol = df_fx_ivol[name_list]

all_data = pd.concat([df_fx_ivol, normal_vol_ts], axis=1)

#------------------------------------------------------------------------------------
LOWER_THRESHOLD = 2
HIGHER_THRESHOLD = 9

mean_rev_ts = mrev.MeanRevTimeSeries(basket)
long_run_mean = mean_rev_ts.get_mean_rev_level()
historical_std = mean_rev_ts.get_stdev()

df_MeanRevStats = mrev.compute_sharpe_and_buysell_signal_multi_period(basket,
                                                                      long_run_mean,
                                                                      historical_std,
                                                                      mean_rev_ts.get_mean_rev_rate_in_days(),
                                                                      mean_rev_ts.get_half_life_in_days(),
                                                                      LOWER_THRESHOLD,
                                                                      HIGHER_THRESHOLD)

df_Sharpe = df_MeanRevStats[['Sharpe Ratio']]

### Time Seires of Basket with z-score using historical std (for trading signal)

In [ ]:
@interact_manual
def show_articles_more_than(start_date = widgets.DatePicker(value=pd.to_datetime('2020-02-01')),
                            end_date   = widgets.DatePicker(value=pd.to_datetime(TODAY)),
                            vert_line_date = widgets.DatePicker(value=pd.to_datetime('2020-04-25')),
                            currency_pairs = name_list):     
    
    column_names = list(all_data.columns)
    
    plot_name_list = [currency_pairs, column_names[-1]]
    
    plot_data = all_data[plot_name_list]
    
    ax = plot_data.plot(secondary_y=[column_names[-1]], kind='line', grid=True, figsize=(15,5), style='-o')
    ax.set_xlim(pd.Timestamp(start_date), pd.Timestamp(end_date))
    
    xlim_range = (start_date, end_date)     
    myplt.normal_plot(df_Sharpe, xlim_range, vert_line_date)     